# Module 2: Run Protein Structure Design and Protein Structure Prediction


NOTE: The authors recommend running this notebook in Amazon SageMaker Studio with the following environment settings:  
* **PyTorch 1.13 Python 3.9 GPU-optimized** image  
* **Python 3** kernel  
* **ml.g4dn.xlarge** instance type  

---

Analyzing large macromolecules like proteins is an essential part of designing new therapeutics. Recently, a number of deep-learning based approaches have improved the speed and accuracy of protein structure analysis. Some of these methods are shown in the image below.

![A diagram of common protein structure design steps](img/01.png)

In this module, we will use several AI algorithms to design **new** variants of the heavy chain for the structure of Herceptin (Trastuzumab). The steps for the pipeline are as follows:

* [RFDiffusion](https://github.com/RosettaCommons/RFdiffusion) is used to generate a small number of variant designs. We will only attempt to redesign parts of the variable region.

* [ProteinMPNN](https://github.com/dauparas/ProteinMPNN) is then used to discover novel sequences that are expected to fold to the novel structure.

* [ESMFold](https://github.com/facebookresearch/esm) is then used to score each of the candidate proteins. ESMFold returns the average predicted local distance difference test (pLDDT) score; which represents the confidence (averaged over all residues) in the predicted structure. This will be used to assess whether the predicted structure is likely to be correct.

For running ESMFold, we will use the ESMFold endpoint deployed in Module 1, so please ensure that you have run that module **before** running this one.

---
## 1. Setup and installation

Install RFDiffusion and it's dependencies

In [ ]:
%pip install -U -q -r protein-design-requirements.txt --disable-pip-version-check

Download and extract the RFDiffusion and ProteinMPNN model weights (This will take several minutes)

In [ ]:
%%bash
mkdir -p "data/weights/rfdiffusion" "data/weights/proteinmpnn" 
aws s3 cp --no-sign-request "s3://aws-batch-architecture-for-alphafold-public-artifacts/compressed/rfdiffusion_parameters_220407.tar.gz" "weights.tar.gz"
tar --extract -z --file="weights.tar.gz" --directory="data/weights/rfdiffusion" --no-same-owner
rm "weights.tar.gz"
wget -q -P "data/weights/proteinmpnn" https://github.com/dauparas/ProteinMPNN/raw/main/vanilla_model_weights/v_48_020.pt
wget -q -P "data" https://files.rcsb.org/download/1N8Z.pdb

---
## 2. Generate new heavy chain structures with RFdiffusion
First we will run RFdiffusion to generate some novel protein structures. To do this, we give the model a starting structure and tell it which which parts to change. We want it to redesign **only** the residues 98-109 on the B chain, while keeping the rest of the structure the same.

Let's take a look at the regions of interest. In the following view, the part of the heavy chain we want to redesign is colored blue and the target region is in green.

![A view of the antibody heavy chain in blue and the target molecule in green](img/herceptin_redesign_target.png)


In [ ]:
import py3Dmol

view = py3Dmol.view(width=600, height=400)
with open("data/1N8Z.pdb") as ifile:
    experimental_structure = "".join([x for x in ifile])
view.addModel(experimental_structure)
view.setStyle({"chain": "A"}, {"opacity": 0})
view.setStyle({"chain": "B"}, {"cartoon": {"color": "blue", "opacity": 0.4}})
view.addStyle(
    {"chain": "B", "resi": "98-109"}, {"cartoon": {"color": "#57C4F8", "opacity": 1.0}}
)
view.setStyle({"chain": "C"}, {"cartoon": {"color": "green", "opacity": 0.4}})
view.setStyle(
    {"chain": "C", "resi": "540-580"}, {"cartoon": {"color": "#37F20E", "opacity": 1.0}}
)
view.zoomTo()
view.show()

Here are our design constraints:

- Look at residues 540-580 of the target molecule (green structure in image above)
- Create a new structure that includes residues 1-97 of the heavy chain (blue above), then generate 10 new residues, then add residues 110-120 from the heavy chain.
- Use residues 570 and 573 from the target molecule as "hotspots", meaning we want to make sure that the new structure interacts with these specific amino acids.
- Create 4 designs in total
- Leave the rest of the RFdiffusion parameters as the default values

The RFDiffusion job will take about 5 minutes to complete on a ml.g4dn.xlarge instance type.

In [ ]:
%%time
!mkdir -p data/results/rfdiffusion

from prothelpers.rfdiffusion import create_structures

create_structures(
    overrides=[
        "inference.input_pdb=data/1N8Z.pdb",
        "inference.output_prefix=data/results/rfdiffusion/rfdiffusion_result",
        "inference.model_directory_path=data/weights/rfdiffusion",
        "contigmap.contigs=[C540-580/0 B1-97/13/B110-120]",
        "ppi.hotspot_res=[C570,C573]",
        "inference.num_designs=4",
    ]
)

Our new designs are in the `data/results/rfdiffusion` folder. Let's take a look at them.

In [ ]:
from prothelpers.structure import extract_structures_from_dir, create_tiled_py3dmol_view

rfdiffusion_results_dir = "data/results/rfdiffusion"
structures = extract_structures_from_dir(rfdiffusion_results_dir)

view = create_tiled_py3dmol_view(structures, total_cols=2)

view.setStyle({"chain": "A"}, {"cartoon": {"color": "blue", "opacity": 0.4}})
view.setStyle(
    {"chain": "A", "resi": "98-109"}, {"cartoon": {"color": "#57C4F8", "opacity": 1.0}}
)
view.setStyle({"chain": "B"}, {"cartoon": {"color": "green"}})

view.zoomTo()
view.show()

## 3. Translate Structure into Sequence with ProteinMPNN
ProteinMPNN is a tool for **inverse protein folding**. In inverse protein folding, the input is a protien tertiary structure, while the output is a sequence (or sequences) that are predicted to fold in the specified structure. Here is a schematic for how it works:
<div style="text-align: left;">
    <img src="img/06.png" alt="A diagram of inverse protein folding" width="700" />
</div>
                        
*image credit: https://huggingface.co/spaces/simonduerr/ProteinMPNN.*        
                               
ProteinMPNN will returns the sequences in [FASTA format](https://software.broadinstitute.org/software/igv/FASTA). Here's an example for the Herceptin Fab antibody that we want to redesign:
```
>1N8Z_2|Chain B|Herceptin Fab (antibody) - heavy chain|Mus musculus (10090)
EVQLVESGGGLVQPGGSLRLSCAASGFNIKDTYIHWVRQAPGKGLEWVARIYPTNGYTRYADSVKGRFTISADTSKN
TAYLQMNSLRAEDTAVYYC**SRWGGDGFYAMDY**WGQGTLVTVSSASTKGPSVFPLAPSSKSTSGGTAALGCLVK
DYFPEPVTVSWNSGALTSGVHTFPAVLQSSGLYSLSSVVTVPSSSLGTQTYICNVNHKPSNTKVDKKVEP
```


We gather the locations of the RFDiffusion output structures and submit them to ProteinMPNN. This will take about 15 seconds on a ml.g4dn.xlarge instance.

In [ ]:
%%time
!mkdir -p data/results/proteinmpnn

from prothelpers import proteinmpnn
from prothelpers.sequence import list_files_in_dir

rfdiffusion_candidates = list_files_in_dir(rfdiffusion_results_dir, extension=".pdb")

proteinmpnn_results_dir = "data/results/proteinmpnn"

for path in rfdiffusion_candidates:
    proteinmpnn.design(
        pdb_path=path,
        out_folder=proteinmpnn_results_dir,
        num_seq_per_target=4,
        pdb_path_chains="A",
        path_to_model_weights="data/weights/proteinmpnn",
        batch_size=1,
        suppress_print=1,
    )

Let's look at the results

In [ ]:
import os
from prothelpers.sequence import extract_seqs_from_dir

mpnn_dir = os.path.join(proteinmpnn_results_dir, "seqs")
mpnn_sequences = extract_seqs_from_dir(mpnn_dir, extension="fa")
print(mpnn_sequences)

## Run Inference on ESMFold

ProteinMPNN has generated 16 new sequences, 4 per predicted structure. But which one should we test in the lab? There are lots of metrics we could use, like isoelectric point, patent status, or homology with other sequences. For this example, we're going to measure the "foldability" of each sequence using ESMFold. This is a popular way to identify those sequences that are most similar to other experimentally-determined structures. Specifically, we'll use the average predicted local distance difference test (pLDDT) score, a measure of the ESMFold prediction confidence.

** NOTE: STOP HERE if you have not finished deploying the ESMFold prediction endpoint from module 1.**

In [ ]:
%store -r endpoint_name

In [ ]:
import boto3

# Wait until ESMFold endpoint from module 1 is in service
waiter = boto3.client('sagemaker').get_waiter('endpoint_in_service')
waiter.wait(EndpointName=endpoint_name)

In [ ]:
%%time
!mkdir -p data/results/esmfold

import json
from prothelpers.structure import get_mean_plddt
import pandas as pd
import sagemaker
from sagemaker.predictor import Predictor

predictor = Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sagemaker.Session(),
    serializer=sagemaker.serializers.CSVSerializer(),
    deserializer=sagemaker.deserializers.StringDeserializer(),
)

metrics = []
for i, seq in enumerate(mpnn_sequences):
    print(f"Generating structure prediction {i} for {seq}")
    esmfold_output = json.loads(predictor.predict(seq))[0]
    mean_plddt = get_mean_plddt(esmfold_output)
    output_file = f"data/results/esmfold/prediction_{i}.pdb"
    with open(output_file, "w") as f:
        f.write(esmfold_output)
    metrics.append(
        {"seq": seq, "esmfold_result": output_file, "mean_plddt": mean_plddt}
    )

metrics_df = (
    pd.DataFrame(metrics)
    .sort_values(by="mean_plddt", ascending=False)
    .reset_index(drop=True)
)
metrics_df

You can see from the results above that the designed proteins have a PLDDT of 0.8 or greater, meaning that ESMFold has high confidence in the structures. The highest-scoring sequences are good candidates for synthesis and testing.

Here is a screenshot of one example of the designed antibody (blue) superimposed on the orignal antibody (green). The orange and red corresponds to the extracellular domain of HER2. Note that the structure of the designed antibody is similair, but not identical to the original.

![Picture of designed protein](img/03.png)


When you are finished with this module, uncomment and run the cell below to delete the ESMFold endpoint.

In [ ]:
%store -z

try:
    predictor.delete_endpoint()
except:
    pass